In [679]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
#import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pdb
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection

scipy: 0.19.1
numpy: 1.13.1
matplotlib: 2.0.2
pandas: 0.20.3
sklearn: 0.18.1


In [282]:
with open('../cspython/esl_teams.pkl', 'rb') as f: 
     d = pkl.load(f)

In [283]:
#with open('../cspython/test.pkl', 'rb') as f: 
#     d = pkl.load(f)

In [470]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [285]:
def combine_dfs(big_data, overview):
    for l in range(len(big_data.keys())):
        series_data = big_data[big_data.keys()[l]]
        series_data_m = merge_matches(series_data)
        series_data_m["date"] = overview.loc[l, 'date']
        series_data_mo = merge_overview(series_data_m, series_data)
        series_data_mos = merge_scoreboards(series_data_mo, series_data)
        series_data_mosm = match_data_board_changer(series_data_mos, series_data)
        if l == 0:
            data = series_data_mosm
            
        else:
            col = ['map','round_num','half','match_id','series_id','ending','CT','T','side_winner','winner','team_A','team_B','team_A_score','team_B_score','match_num','team_players','K-D','+/-','ADR','KAST','Rating2.0','nicknames']
            data = pd.concat([data, series_data_mosm])  
            col2 = data.columns
            data = data[col + list(set(col2) - set(col))]
    data = data.reset_index()
    return data

In [286]:
def merge_matches(series_data):
    for d in range(0,len(series_data['matches'])):
        if d == 0:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = series_data['matches'][d]
        else:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = pd.concat([series_data_m, series_data['matches'][d]])
    
    return series_data_m
    #should work for concact the matches together


In [287]:
def merge_overview(series_data_m, series_data):
    series_data['match_overview']['team_A_name'] = series_data['match_overview'].columns[4]
    series_data['match_overview']['team_B_name'] = series_data['match_overview'].columns[5]
    
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] == series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_B_name
    series_data['match_overview'].loc[(series_data['match_overview']['winner'] != series_data['match_overview'].columns[4]),'loser_of_match'] = series_data['match_overview'].team_A_name
    series_data['match_overview'] = series_data['match_overview'].rename(index = str, columns ={series_data['match_overview'].columns[3]: "winner_of_match",series_data['match_overview'].columns[4]: "team_A_score",series_data['match_overview'].columns[5]: "team_B_score"})
    
    
    series_data_mo = pd.merge(series_data_m, series_data['match_overview'], on=['match_id', 'map', 'series_id'])
    
    return series_data_mo
#works at merging matches with match_overviewb

In [288]:
def merge_scoreboards(series_data_mo, series_data):    
   
    for i in range(len(series_data['scoreboards'][0])):
        series_data['scoreboards'][0][i]['match_num'] = i+1
        #pdb.set_trace()
        series_data['scoreboards'][0][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_A_name'].unique()[0]
        series_data['scoreboards'][0][i] = series_data['scoreboards'][0][i].rename(index = str, columns={ series_data['scoreboards'][0][i].columns[0] : "team_players" })
        
        series_data['scoreboards'][1][i]['match_num'] = i+1
        series_data['scoreboards'][1][i]['player_team_name'] = series_data_mo.loc[(series_data_mo['match_num']== i+1),'team_B_name'].unique()[0]
        series_data['scoreboards'][1][i] = series_data['scoreboards'][1][i].rename(index = str, columns={ series_data['scoreboards'][1][i].columns[0] : "team_players"})
        
        new_df = pd.concat([series_data['scoreboards'][0][i], series_data['scoreboards'][1][i]])
        
        if i == 0:
            con_df = new_df
        else:
            con_df = pd.concat([con_df, new_df])
   
    series_data_mos = pd.merge(series_data_mo, con_df, how='outer', on='match_num')
    series_data_mos['nicknames'] = series_data_mos['team_players'].str.split(expand = True)[3]
    return series_data_mos

       
        
#works at adding match_num to scoreboards
        
                         

In [289]:
def match_data_board_changer(series_data_mos,series_data):
    
    board_name = ['first_kills','who_kill_who', 'awp_kills']

    for idx, a in enumerate(series_data['match_data']):
        new_df = pd.DataFrame()
        for idx1, c in enumerate(board_name):    
            new_board_c = pd.DataFrame()
            new_board_r= pd.DataFrame()
            names_c = a[c].set_index('Unnamed: 0').columns
            for b in names_c:
                new_board_c[b+'_'+c] = a[c].set_index('Unnamed: 0')[b].str.split(pat = ':', expand = True)[0]
               
            names_r = a[c].set_index('Unnamed: 0').T.columns
            for b in names_r:
                new_board_r[b+'_'+c] = a[c].set_index('Unnamed: 0').T[b].str.split(pat = ':', expand = True)[1]
            new_board_c['nicknames'] = new_board_c.index
            new_board_r['nicknames'] = new_board_r.index
            
            board_df = new_board_c.append(new_board_r)
            
            if idx1 == 0:
                new_df = board_df
            else:
                new_df = pd.merge(new_df, board_df, on = 'nicknames')
        if idx == 0:
            new_df['match_num'] = 1+idx
            con_df = new_df
        else:
            new_df['match_num'] = 1+idx
            try:
                con_df = con_df.append(new_df, ignore_index=True)
            except:
                print con_df.columns
                print new_df.columns
    con_df = con_df.loc[:, ~con_df.columns.duplicated()]
    #pdb.set_trace()
  
    series_data_mosm = pd.merge(series_data_mos, con_df, on=['nicknames','match_num'])
    return series_data_mosm


In [471]:
#data = combine_dfs(big_data, overview)

In [458]:
def create_fwa_dr_columns(data, col_list):  # first k , awp, who, divided by rounds 
    columns = pd.Series(data.columns)
    for a in col_list:
        col = columns[columns.str.contains(a)]
        data[a+'_sum_dr'] = data[col].convert_objects(convert_numeric = True).sum(axis = 1) / (data['team_A_score'] + data['team_B_score'])
    data.loc[:, data.columns != 'date'] = data.loc[:, data.columns != 'date'].apply(pd.to_numeric, errors='ignore')
    return data
        
    

In [535]:
def create_fwadr_his(data, col_list):   # column with historic awp, first ,who_killwho of player vs player.
    match_id = data.match_id.unique()
    players = data.nicknames.unique()
    for a in col_list:
        grouping = data.groupby(['nicknames', 'match_id'])[a+'_sum_dr'].mean() 
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping = grouping.groupby(['nicknames'])[a+'_sum_dr'].sum()/ grouping.groupby(['nicknames'])[a+'_sum_dr'].count()
        grouping = pd.DataFrame(grouping)
        grouping = grouping.reset_index()
        grouping = grouping.rename(index=str, columns={a +'_sum_dr': a + '_sum_dr_hist'})
        data = pd.merge(data, grouping, on = 'nicknames')
        
    return data
    

In [531]:
def create_matches_count(data): # how many matches a person has played
    grouping = data.groupby(['nicknames', 'match_id'])['ADR'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.reset_index()
    grouping = grouping.groupby(['nicknames'])['ADR'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.reset_index()
    grouping = grouping.rename(index=str, columns={'ADR': 'matches_played_player'})
    data = pd.merge(data, grouping, on = 'nicknames')
    return data
    
    

In [461]:
# how many times a person has played a map.

In [536]:
def create_avdamage_his(data):  # column with historic average damage of individual 
    grouping = data.groupby(['nicknames', 'match_id'])['ADR'].mean() 
    grouping = pd.DataFrame(grouping)
    grouping = grouping.reset_index()
    grouping = grouping.groupby(['nicknames'])['ADR'].sum()/ grouping.groupby(['nicknames'])['ADR'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.reset_index()
    grouping = grouping.rename(index=str, columns={'ADR': 'ADR_hist'})
    data = pd.merge(data, grouping, on = 'nicknames')
    
    return data
    

In [463]:
def create_avdamage_map_his(data):# historic average damage of individual for each map
    map_adr = data.groupby(['match_id','nicknames', 'map'])['ADR'].mean()
    map_adr = pd.DataFrame(map_adr)
    map_adr = map_adr.reset_index()
    map_adr = map_adr.groupby(['nicknames', 'map'])['ADR'].mean()
    map_adr = pd.DataFrame(map_adr)
    map_adr = map_adr.reset_index()
    for a in list(map_adr.map.unique()):
        map_adr.loc[:,'ADR_his_'+ a] = 0
        map_adr.loc[(map_adr.loc[:, 'map'] == a), 'ADR_his_'+ a] = map_adr.loc[:,'ADR']
    map_adr = map_adr.drop('ADR', axis = 1)
    data = pd.merge(data, map_adr, on =['nicknames', 'map'])
    return data
    

In [464]:
def create_faw_map_his(data, col_list):
    for a in col_list:
        fk_map = data.groupby(['match_id','nicknames', 'map'])[a + '_sum_dr'].mean()
        fk_map = pd.DataFrame(fk_map)
        fk_map = fk_map.reset_index()
        fk_map = fk_map.groupby(['nicknames', 'map'])[a + '_sum_dr'].mean()
        fk_map = pd.DataFrame(fk_map)
        fk_map = fk_map.reset_index()
        for b in list(fk_map.map.unique()):
            fk_map.loc[:,a+'_'+b+'_dr_hist'] = 0 
            fk_map.loc[(fk_map.loc[:, 'map'] == b), a+'_'+b+'_dr_hist'] = fk_map.loc[:,a + '_sum_dr']
        fk_map = fk_map.drop(a + '_sum_dr', axis = 1)
        data = pd.merge(data, fk_map, on = ['nicknames', 'map'])
    return data
    

In [465]:
def create_map_win_loss_his(data):  # team total win and loses on map with total times played on map
    maps = data.map.unique()
    teams = data.player_team_name.unique()
    match_id = data.match_id.unique()
    for a in maps:
        data[a + "_win_his"] = 0
        data[a + "_loss_his"] = 0
        data[a + "_total_played"] = 0
    for a in match_id:
        map_for_match = data.loc[(data['match_id'] == a) ,'map'].unique()
        winner_of_map = data.loc[(data['match_id'] == a), 'winner_of_match'].unique()
        loser_of_map = data.loc[(data['match_id'] == a), 'loser_of_match'].unique()
        data.loc[(data['player_team_name'] == winner_of_map[0]), [map_for_match[0] + "_win_his", map_for_match[0] +'_total_played']] += 1
        data.loc[(data['player_team_name'] == loser_of_map[0]), [map_for_match[0] + "_loss_his", map_for_match[0] +'_total_played']] += 1 
       
    return data

In [466]:
def create_map_win_his_per(data): #percentage team total win and loses on map
    teams = data.player_team_name.unique()
    maps = data.map.unique()
    for a in maps:
        data[a + '_win_perc_map'] = 0
        for b in teams:
            pg = (data.player_team_name == b) 
            data.loc[pg,a + '_win_perc_map'] = data.loc[pg, a + "_win_his"].unique()[0] / (data.loc[pg, a + "_win_his"].unique()[0] + data.loc[pg, a + "_loss_his"].unique()[0]) 
            data.loc
    return data    
    
    

In [524]:
def create_rounds_won_vs_team_his(data): # team rounds won vs another team
    grouping = data.groupby(['match_id','map','team_A_name','team_B_name', 'team_A_score', 'team_B_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['team_A_name', 'team_B_name']).sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    forward = grouping.team_A_name+grouping.team_B_name
    reverse = grouping.team_B_name+grouping.team_A_name
    for idx, val in enumerate(forward):
        for idx2, val2 in enumerate(reverse):
            if val == val2 and idx < idx2:
                grouping.loc[idx,'team_A_score_Count'] += grouping.loc[idx2,'team_B_score_Count']
                grouping.loc[idx,'team_B_score_Count'] += grouping.loc[idx2,'team_A_score_Count']
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
            elif val == val2 and idx > idx2:
                grouping.loc[idx2,'team_B_score_Count'] = grouping.loc[idx,'team_A_score_Count']
                grouping.loc[idx2,'team_A_score_Count'] = grouping.loc[idx,'team_B_score_Count']
    
    grouping = grouping.drop('round_num_Count_Count', axis = 1)
    col1 = list(grouping.team_A_name.unique())
    col2  = list(grouping.team_B_name.unique())
    col = col1 + col2
    col = list(set(col))
    data = pd.merge(data,grouping, on=['team_A_name', 'team_B_name']) 
    for a in col:
        data['rd_total_his_'+ a] = 0
        data.loc[(data.player_team_name != a) & (data.team_A_name == a) , 'rd_total_his_'+ a]=data.team_B_score_Count
        data.loc[(data.player_team_name != a) & (data.team_B_name == a) , 'rd_total_his_'+ a]=data.team_A_score_Count
    
    return data    

In [468]:
def create_total_team_rd_map_his(data):
    grouping = data.groupby(['map','team_A_name', 'team_A_score'])['round_num'].count()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    grouping = grouping.groupby(['map','team_A_name'])[ 'team_A_score'].sum()
    grouping = pd.DataFrame(grouping)
    grouping = grouping.add_suffix('_Count').reset_index()
    fgrouping = data.groupby(['player_team_name','map','team_B_name', 'team_B_score'])['round_num'].count()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.groupby(['map','team_B_name'])[ 'team_B_score'].sum()
    fgrouping = pd.DataFrame(fgrouping)
    fgrouping = fgrouping.add_suffix('_Count').reset_index()
    fgrouping = fgrouping.rename(index=str, columns={"team_B_name": "team_A_name", 'team_B_score_Count': 'team_A_score_Count'})
    merged = pd.concat([grouping, fgrouping], axis = 0)
    merged.groupby(['map', 'team_A_name'])['team_A_score_Count'].sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index(drop = True)
    merged = merged.rename(index=str, columns={"team_A_name": "player_team_name", "team_A_score_Count": 'total_team_rd_map'})
    merged = merged.groupby(['player_team_name', 'map']).sum()
    merged = pd.DataFrame(merged)
    merged = merged.reset_index()
    for a in list(merged.map.unique()):
        merged.loc[:,'total_team_rd_'+ a] = 0
        merged.loc[(merged.loc[:, 'map'] == a), 'total_team_rd_'+ a] = merged.loc[:,'total_team_rd_map']
    merged = merged.drop('total_team_rd_map', axis = 1)
    data = pd.merge(data, merged, on =['player_team_name', 'map'])
    return data
    

In [472]:
#data.to_pickle("data_converged.pkl")

In [537]:
data = pd.read_pickle("data_converged.pkl")

In [538]:
col_list = ['first_kills', 'who_kill_who','awp_kills']
data = create_fwa_dr_columns(data, col_list)
print 1
data = create_fwadr_his(data,col_list)
print 2
data = create_avdamage_his(data)
print 3
data = create_map_win_loss_his(data)
print 4
data = create_map_win_his_per(data)
print 5
data = create_rounds_won_vs_team_his(data)
print 6
data = create_total_team_rd_map_his(data)
print 7
data = create_avdamage_map_his(data)
print 8
data = create_faw_map_his(data, col_list)
print 9
data = create_matches_count(data)

1
2
3
4
5
6
7
8
9


In [694]:
data_adv = data.loc[:, 'first_kills_sum_dr_hist':'matches_played_player']

In [695]:
data_adv['match_id'] = data['match_id']

In [696]:
data_adv['player_team_name'] = data['player_team_name']

In [697]:
data_adv = data_adv.drop(['team_A_score_Count','team_B_score_Count'], axis = 1)

In [698]:
data_adv = data_adv.groupby(['match_id', 'player_team_name']).sum()

In [699]:
data_adv

first_kills_sum_dr_hist  \
match_id                             player_team_name                            
002f2963-ccea-4811-a7b9-d4182ae1daad CLG                             14.063025   
                                     Immortals                       11.881182   
0083a7a7-c355-4e73-bc40-66e74dfff72c Luminosity                      14.454737   
                                     NRG                             13.794940   
00df5d72-7a8d-4593-9632-aa0fba4de857 Immortals                       11.942878   
                                     Rogue                           13.915219   
031a3ebd-1884-441e-938b-0bbaf97c690d Cloud9                          14.600350   
                                     SK                              11.750112   
036af896-a5e2-406f-9964-e46d3d704d2c Misfits                         11.107916   
                                     NRG                             11.331558   
039fd919-a814-4a78-8283-9c3d8910da2a CLG                             15.067527   
                                     NRG                             14.780293   
04119411-5ce8-4bfa-b1ef-4f7e2e593ccf Immortals                       11.552376   
                                     Luminosity                      10.841053   
045e3878-d3cb-48a2-8d7c-8f1dcd9e8627 CLG                             15.067527   
                                     Liquid                          13.053067   
052cff4f-8970-46cc-b8ef-ef70436e785e Liquid                           9.137147   
                                     SK                               9.138976   
058d4dc9-0df0-4aa5-bda1-ac4a7ec23349 Ghost                            9.754200   
                                     Liquid                          10.877556   
06441a60-5901-4a63-ad7e-8181f4d7d8de CLG                             12.054021   
                                     Misfits                         11.590869   
0651429a-40b1-412c-8ab3-6ed12966f688 Renegades                       12.664868   
                                     compLexity                      11.921762   
06d5e801-4d14-406d-bafe-3773c02a0022 CLG                             14.063025   
                                     Renegades                       14.478927   
071cc712-74e9-48f8-b9db-ee50e1d1f1ad NRG                             14.287616   
                                     compLexity                      14.101081   
07343f1f-e5c0-4630-9f9f-a068cf9ddf22 Misfits                         10.142011   
                                     NRG                             10.346205   
...                                                                        ...   
f8dc36c5-32b6-41ec-b64f-b76c1352c516 Liquid                           9.137147   
                                     Misfits                         10.142011   
f8e9eab0-150f-4623-ae76-e2d3b869c1b1 Ghost                           10.144368   
                                     OpTic                           14.387704   
f91a00f7-7289-4d33-91bf-95772c099c1d CLG                             15.067527   
                                     compLexity                      14.587325   
f95a41c3-052b-4a5b-b0a8-49079f96b6ed Liquid                          10.877556   
                                     OpTic                           13.202793   
fb42bf41-8a08-4b52-a00b-ccdfa2e0e69b Luminosity                      13.422256   
                                     Renegades                       12.984546   
fb9f8029-8fdd-4f75-b722-3ba848e39a06 Liquid                          12.470261   
                                     NRG                             16.012016   
fc9fecbe-f581-4aed-a719-a011b1913276 CLG                             12.556272   
                                     Misfits                         11.553616   
fcd65134-150e-4729-aff5-c02a50dd9e9b Immortals                        7.346696   
                                     Liquid                           7.396738   
fdcf7951-c6af-44c4-a123-6f50b5962255 

In [700]:
data_adv = pd.DataFrame(data_adv)
data_adv = data_adv.reset_index()

In [701]:
pd.set_option('display.max_columns', 300)

In [702]:
data_adv.shape

(1208, 84)

In [703]:
data_adv = data_adv.apply(pd.to_numeric, errors='ignore')

In [704]:
datay = data.loc[:,['winner_of_match', 'player_team_name', 'match_id']]

In [705]:
datay.loc[datay.winner_of_match != datay.player_team_name, 'winner_of_match'] = 0
datay.loc[datay.winner_of_match == datay.player_team_name, 'winner_of_match'] = 1


In [706]:
datay.winner_of_match = datay.winner_of_match.apply(pd.to_numeric, errors='ignore')

In [707]:
datay = datay.groupby(['match_id', 'player_team_name'])['winner_of_match'].mean()

In [708]:
datay = pd.DataFrame(datay)
datay = datay.reset_index()

In [709]:
datay

,match_id,player_team_name,winner_of_match
0,002f2963-ccea-4811-a7b9-d4182ae1daad,CLG,1
1,002f2963-ccea-4811-a7b9-d4182ae1daad,Immortals,0
2,0083a7a7-c355-4e73-bc40-66e74dfff72c,Luminosity,0
3,0083a7a7-c355-4e73-bc40-66e74dfff72c,NRG,1
4,00df5d72-7a8d-4593-9632-aa0fba4de857,Immortals,0
5,00df5d72-7a8d-4593-9632-aa0fba4de857,Rogue,1
6,031a3ebd-1884-441e-938b-0bbaf97c690d,Cloud9,0
7,031a3ebd-1884-441e-938b-0bbaf97c690d,SK,1
8,036af896-a5e2-406f-9964-e46d3d704d2c,Misfits,0
9,036af896-a5e2-406f-9964-e46d3d704d2c,NRG,1


In [710]:
datay = datay.drop(['player_team_name','match_id'], axis = 1)

In [711]:
datay.shape

(1208, 1)

In [712]:
data_adv = data_adv.drop(['match_id', 'player_team_name'], axis = 1)

In [713]:
data_adv.shape

(1208, 82)

In [714]:
y = datay.values.astype(int)
X = data_adv.values

In [715]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'

In [716]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [717]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'


In [718]:
models = []
models.append(('LR', LogisticRegression()))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('NB', GaussianNB()))
models.append(('CART', DecisionTreeClassifier())) 
models.append(('KNeighborsClassifier', KNeighborsClassifier())) 
#models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('SVC', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(shuffle = True, n_splits=num_folds, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.671357 (0.036968)
LDA: 0.668905 (0.036129)
NB: 0.577775 (0.037021)
CART: 0.545448 (0.036477)
KNeighborsClassifier: 0.549587 (0.044508)
GradientBoostingClassifier: 0.596811 (0.056151)
AdaBoostClassifier: 0.620833 (0.043225)
RandomForestClassifier: 0.552975 (0.026475)
ExtraTreesClassifier: 0.561219 (0.029005)
DecisionTreeClassifier: 0.548822 (0.027753)
SVC: 0.472741 (0.036530)
